In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymsm.plotting import stackplot
from pymsm.simulation import MultiStateSimulator, default_update_covariates_function

%load_ext autoreload
%autoreload 2

# Rossi - single state example

In [3]:
# Load rossi dataset
from lifelines.datasets import load_rossi

rossi = load_rossi()
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(rossi, duration_col="week", event_col="arrest")
baseline_hazard = cph.baseline_hazard_["baseline hazard"]
coefs = cph.params_

from pymsm.datasets import load_rossi_competing_risk_data

rossi_competing_risk_data, covariate_names = load_rossi_competing_risk_data()

### Define the full model ###
competing_risks_models_list = [
    {
        "origin_state": 1,
        "target_states": [2],
        "model_defs": [{"coefs": coefs, "baseline_hazard": baseline_hazard}],
    }
]

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[2],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_names,
)

# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=rossi_competing_risk_data.loc[0, covariate_names],
    origin_state=1,
    current_time=0,
    n_random_samples=3,
    max_transitions=10,
    print_paths=True,
)


100%|██████████| 3/3 [00:00<00:00, 899.10it/s]

[1, 2]
[25.0]
[1, 2]
[15.0]
[1, 2]
[11.0]


# 2 state example - AIDS-SI

In [4]:
from pymsm.datasets import load_aidssi

data = load_aidssi()
data.head()


# TODO 


,patnr,time,status,cause,ccr5
1,1,9.106,1,AIDS,WW
2,2,11.039,0,event-free,WM
3,3,2.234,1,AIDS,WW
4,4,9.878,2,SI,WM
5,5,3.819,1,AIDS,WW


# Many-state example - EBMT

In [5]:
# Fit model from data and extract model parts for simulation

from pymsm.datasets import prep_ebmt_long
competing_risk_dataset, covariate_cols, states_labels = prep_ebmt_long()

from pymsm.multi_state_competing_risks_model import MultiStateModel
multi_state_model = MultiStateModel(
    dataset=competing_risk_dataset,
    terminal_states = [5, 6],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_cols,
    competing_risk_data_format=True,
)

multi_state_model.fit()


Fitting Model at State: 1
>>> Fitting Transition to State: 2, n events: 785
>>> Fitting Transition to State: 3, n events: 907
>>> Fitting Transition to State: 5, n events: 95
>>> Fitting Transition to State: 6, n events: 160
Fitting Model at State: 2
>>> Fitting Transition to State: 5, n events: 112
>>> Fitting Transition to State: 6, n events: 39
>>> Fitting Transition to State: 4, n events: 227
Fitting Model at State: 3
>>> Fitting Transition to State: 4, n events: 433
>>> Fitting Transition to State: 6, n events: 197
>>> Fitting Transition to State: 5, n events: 56
Fitting Model at State: 4
>>> Fitting Transition to State: 5, n events: 107
>>> Fitting Transition to State: 6, n events: 137


In [9]:
from pymsm.simulation import extract_competing_risks_models_list_from_msm

competing_risks_models_list = extract_competing_risks_models_list_from_msm(
    multi_state_model, verbose=True
)

# Configure the simulator
mssim = MultiStateSimulator(
    competing_risks_models_list,
    terminal_states=[5,6],
    update_covariates_fn=default_update_covariates_function,
    covariate_names=covariate_cols,
)

# Run simulation
mc_paths = mssim.run_monte_carlo_simulation(
    sample_covariates=competing_risk_dataset.loc[0, covariate_cols],
    origin_state=1,
    current_time=0,
    n_random_samples=5,
    max_transitions=10,
    print_paths=True,
)


1 2 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff153f6ac10>
1 3 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13d3fcd00>
1 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13d442ac0>
1 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13c3e0e20>
2 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b078340>
2 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b078280>
2 4 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b09ad60>
3 4 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b82c6a0>
3 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b091d00>
3 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b7f1160>
4 5 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b071a90>
4 6 <pymsm.competing_risks_model.EventSpecificModel object at 0x7ff13b824be0>


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [08:59<?, ?it/s]


KeyError: 2

In [12]:
mssim.state_specific_models[1].event_specific_models

{6: <pymsm.competing_risks_model.EventSpecificModel at 0x7ff13ba1ec40>}

In [17]:
len(competing_risks_models_list[0]['model_defs'])

4

In [25]:
s = []
for i in range(1_000_000):
    s.append(np.sqrt(i))

In [27]:
from joblib import Parallel, delayed
from multiprocessing import Manager

manager = Manager()
s = manager.list()

def run(v, s):
    s.append(v)
    return

_ = Parallel(n_jobs=4)(delayed(run)(v, s) for v in range(10))

TypeError: 'NoneType' object is not callable